In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras

from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb

# Data Preprocessing

In [7]:
vocab_size = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size) # most frequent 20000 words
print(X_train.shape, y_train.shape)
print(X_train[0])

(25000,) (25000,)
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
# use padding
avg_train_X = np.mean(list(map(len, X_train)))
avg_test_X = np.mean(list(map(len, X_test)))
print(avg_train_X, avg_test_X)

max_len = 400
pad_train_X = pad_sequences(X_train, maxlen=max_len)
pad_test_X = pad_sequences(X_test, maxlen=max_len)
print(pad_train_X.shape, pad_test_X.shape)


238.71364 230.8042
(25000, 400) (25000, 400)


# Modeling

In [9]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [10]:
"""
 using embedding layer without additional layers
"""
embedding_dims=64

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dims, input_length=max_len))
model.add(GlobalAveragePooling1D()) # average of all word vectors
model.add(Dense(1, activation='sigmoid')) # output layer
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(pad_train_X, y_train, batch_size=32, epochs=10, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 64)           1280000   
                                                                 
 global_average_pooling1d (  (None, 64)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1280065 (4.88 MB)
Trainable params: 1280065 (4.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6295 - accuracy: 0.7304 - val_loss: 0.5218 - val_accuracy: 0.8206
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 

In [11]:
# save model
model_path = "../model/"
model.save(model_path + 'imdb_model.h5')

/Users/godpeny/Code/venv/laboratory/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
loaded_model = load_model(model_path + 'imdb_model.h5')
print(loaded_model.evaluate(pad_test_X, y_test)) # loss, accuracy

782/782 [==============================] - 1s 1ms/step - loss: 0.2910 - accuracy: 0.8875
[0.29098179936408997, 0.8874800205230713]
